In [1]:
import sys
import os

# Thêm đường dẫn của thư mục cha vào sys.path
sys.path.append(os.path.abspath(".."))

# 1. Xem file .txt

In [2]:
filePath = r"d:\13_21133021_NguyenTrongDung_LuuTruMonHoc\data_KLTN\dataset_Query.txt"

# Mở và đọc file
with open(filePath, "r", encoding="utf-8") as file:
    lines = [next(file) for _ in range(10)]  # Đọc 10 dòng đầu tiên

# Hiển thị nội dung
for i, line in enumerate(lines, 1):
    print(f"{line.strip()}")

Theo quy định, chuẩn chương trình đào tạo đại học bao gồm những yếu tố nào?
Chuẩn đầu ra của chương trình đào tạo được xác định dựa trên những tiêu chí nào?
Khối lượng học tập tối thiểu cho một chương trình đào tạo đại học được quy định như thế nào?
Chương trình đào tạo định hướng nghiên cứu khác gì so với chương trình đào tạo định hướng ứng dụng?
Những yêu cầu nào được đặt ra đối với đội ngũ giảng viên giảng dạy ở trình độ đại học?
Hội đồng tư vấn khối ngành có nhiệm vụ gì trong quá trình xây dựng chuẩn chương trình đào tạo?
Việc cập nhật chuẩn chương trình đào tạo cần được thực hiện bao lâu một lần?
Điều kiện để một cá nhân được công nhận là nghiên cứu sinh trình độ tiến sĩ tại Việt Nam là gì?
Chương trình đào tạo tiến sĩ có bắt buộc phải bao gồm nghiên cứu khoa học không? Nếu có, vì sao?
Quy trình phản biện độc lập luận án tiến sĩ được thực hiện như thế nào?


# 2. Thực thi tổng quát

In [3]:
import os
import time
import pandas as pd

from dotenv import load_dotenv
from source.Gemini.apikeys_gemini import APIKeyManager
from source.Gemini.gemini import Gemini

In [4]:
# Load API keys
load_dotenv()
MODEL_GEMINI = os.getenv("MODEL_GEMIMI")
APIS_GEMINI_LIST = os.getenv('APIS_GEMINI_LIST').split(',')
key_manager = APIKeyManager(APIS_GEMINI_LIST)
model_gemini = Gemini(key_manager, MODEL_GEMINI)

In [5]:
# Đọc file dataset_Query.txt
file_path = r"d:\13_21133021_NguyenTrongDung_LuuTruMonHoc\data_KLTN\dataset_Query.txt"
output_file = r"d:\13_21133021_NguyenTrongDung_LuuTruMonHoc\data_KLTN\dataset_Query_Rewritting.xlsx"

In [6]:
# Số dòng cần tích lũy trước khi ghi vào file
batch_size = 100
data_to_save = []

In [7]:
def get_current_row_count(file_path):
    if os.path.exists(file_path):
        try:
            df = pd.read_excel(file_path, engine="openpyxl")
            return len(df)
        except Exception:
            return 0
    return 0

In [8]:
with open(file_path, "r", encoding="utf-8") as file:
    for line_num, original_query in enumerate(file, start=1):
        original_query = original_query.strip()  # Xóa khoảng trắng

        if not original_query:  # Bỏ qua dòng trống
            continue

        # Gọi model để tạo 5 câu truy vấn mới
        queries = model_gemini.query_generator(original_query)

        # Lưu vào danh sách
        data_to_save.append(queries)

        # Đợi 5 giây để tránh nghẽn API
        time.sleep(5)

        # Khi đủ batch_size (100 dòng), ghi vào file
        if len(data_to_save) >= batch_size:
            df_new = pd.DataFrame(data_to_save, columns=["Original Query", "Query 1", "Query 2", "Query 3", "Query 4", "Query 5"])

            # Kiểm tra nếu file đã tồn tại thì đọc và nối dữ liệu
            if os.path.exists(output_file):
                try:
                    df_existing = pd.read_excel(output_file, engine="openpyxl")
                    df_final = pd.concat([df_existing, df_new], ignore_index=True)
                except Exception:
                    df_final = df_new  # Nếu lỗi, chỉ ghi dữ liệu mới
            else:
                df_final = df_new  # Nếu file chưa tồn tại, chỉ ghi dữ liệu mới

            # Ghi dữ liệu vào file Excel
            df_final.to_excel(output_file, index=False, engine="openpyxl")

            # Lấy số dòng hiện tại trong file
            current_row_count = get_current_row_count(output_file)
            print(f"Đã lưu dữ liệu, tổng số dòng hiện tại trong .xlsx: {current_row_count}")

            data_to_save = []  # Reset danh sách để tiếp tục

Đã lưu dữ liệu, tổng số dòng hiện tại trong .xlsx: 100
Đã lưu dữ liệu, tổng số dòng hiện tại trong .xlsx: 200
Đã lưu dữ liệu, tổng số dòng hiện tại trong .xlsx: 300
Đã lưu dữ liệu, tổng số dòng hiện tại trong .xlsx: 400
Đã lưu dữ liệu, tổng số dòng hiện tại trong .xlsx: 500
Đã lưu dữ liệu, tổng số dòng hiện tại trong .xlsx: 600
Đã lưu dữ liệu, tổng số dòng hiện tại trong .xlsx: 700
Đã lưu dữ liệu, tổng số dòng hiện tại trong .xlsx: 800
Đã lưu dữ liệu, tổng số dòng hiện tại trong .xlsx: 900
Đã lưu dữ liệu, tổng số dòng hiện tại trong .xlsx: 1000


In [9]:
# Ghi phần còn lại nếu chưa đủ batch_size nhưng vẫn có dữ liệu
if data_to_save:
    df_new = pd.DataFrame(data_to_save, columns=["Original Query", "Query 1", "Query 2", "Query 3", "Query 4", "Query 5"])

    if os.path.exists(output_file):
        try:
            df_existing = pd.read_excel(output_file, engine="openpyxl")
            df_final = pd.concat([df_existing, df_new], ignore_index=True)
        except Exception:
            df_final = df_new
    else:
        df_final = df_new

    df_final.to_excel(output_file, index=False, engine="openpyxl")

    # Lấy số dòng hiện tại trong file
    current_row_count = get_current_row_count(output_file)
    print(f"Đã lưu dữ liệu, tổng số dòng hiện tại trong .xlsx: {current_row_count}")

Đã lưu dữ liệu, tổng số dòng hiện tại trong .xlsx: 1075
